In [198]:
# import packages
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.model_selection import cross_validate
import sklearn.metrics as metrics
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

In [199]:
df = pd.read_csv("./Data/all-months-divvy-tripdata.csv")

C:\Users\jakee\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [200]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,s,Unnamed: 13,duration,distance,start_hour,end_hour,temp,precip,humid,snow_depth
0,0,B9F73448DFBE0D45,classic_bike,1/24/2021 19:15,1/24/2021 19:22,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,member,NaN,NaN,7.0,1.266101,19.250000,19.366667,33.5,0.0,69.21,0.00
1,1,457C7F4B5D3DA135,electric_bike,1/23/2021 12:57,1/23/2021 13:02,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,-87.696733,41.910435,-87.696890,member,NaN,NaN,5.0,0.692238,12.950000,13.033333,26.6,0.0,41.83,0.00
2,2,57C750326F9FDABE,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,-87.696688,41.899180,-87.672178,casual,NaN,NaN,9.0,1.266479,15.466667,15.616667,37.5,0.0,55.02,0.39
3,3,4D518C65E338D070,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,-87.696716,41.899149,-87.672177,casual,NaN,NaN,9.0,1.268119,15.466667,15.616667,37.5,0.0,55.02,0.39
4,4,9D08A3AFF410474D,classic_bike,1/24/2021 15:56,1/24/2021 16:07,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,casual,NaN,NaN,11.0,1.266101,15.933333,16.116667,34.1,0.0,65.50,0.00


In [201]:
def id_fix(id_n):
    return id_n if isinstance(id_n, int) or id_n.isdigit() else 0
    
# df["start_station_id"][0].type()

df["start_station_id"] = df["start_station_id"].apply(id_fix)

type(df["start_station_id"][0])

str

In [202]:
def mem_cas(mem):
    return mem == "member"

df["mem_cas_bool"] = df["member_casual"].apply(mem_cas)

In [203]:
from sklearn.preprocessing import KBinsDiscretizer

# est = KBinsDiscretizer(n_bins=20, encode='ordinal', strategy='uniform')
# df["bin_station"] = est.fit_transform(np.array(df["start_station_id"]).reshape(-1, 1))
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')

df["bin_lat"] = est.fit_transform(np.array(df["start_lat"]).reshape(-1, 1))
df["bin_lng"] = est.fit_transform(np.array(df["start_lng"]).reshape(-1, 1))


In [204]:
def rideTypeToInt(rType):
    if rType == 'classic_bike':
        return 0
    elif rType == 'electric_bike':
        return 1
    else:
        return 2
    
df["ride_type_int"] = df["rideable_type"].apply(rideTypeToInt)


In [205]:
from sklearn.preprocessing import KBinsDiscretizer

est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
df["temp_bins"] = est.fit_transform(np.array(df["temp"]).reshape(-1, 1))

In [206]:
df['datetime'] = df.apply(lambda x : x['started_at'].split(' ')[0], axis=1)
df['datetime'] = df['datetime'].apply(lambda x : x.split('/')[2] + '-' + ('0' + x.split('/')[0])[-2:] + '-' + ('0' + x.split('/')[1])[-2:])
ride_date_weekend_bool = {} # key: ride_date, value: 1 if the date is a Saturday, Sunday, or holiday, else 0
holidays = ['2020-01-01', '2020-01-20', '2020-05-25', '2020-07-03', '2020-09-07', '2020-11-11', '2020-11-26', '2020-12-25',
            '2021-01-01', '2020-01-18', '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-11', '2021-11-25', '2021-12-24', '2021-12-31']

def is_wkd(date):
    current_timestamp = pd.Timestamp(date)
    return current_timestamp.dayofweek in [5,6] or row['datetime'] in holidays


df['weekend_or_holiday'] = df["datetime"].apply(is_wkd)
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,s,Unnamed: 13,duration,distance,start_hour,end_hour,temp,precip,humid,snow_depth,mem_cas_bool,bin_lat,bin_lng,ride_type_int,temp_bins,datetime,weekend_or_holiday
0,0,B9F73448DFBE0D45,classic_bike,1/24/2021 19:15,1/24/2021 19:22,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,member,NaN,NaN,7.0,1.266101,19.250000,19.366667,33.5,0.0,69.21,0.00,True,6.0,3.0,0,1.0,2021-01-24,True
1,1,457C7F4B5D3DA135,electric_bike,1/23/2021 12:57,1/23/2021 13:02,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,-87.696733,41.910435,-87.696890,member,NaN,NaN,5.0,0.692238,12.950000,13.033333,26.6,0.0,41.83,0.00,True,6.0,3.0,1,1.0,2021-01-23,True
2,2,57C750326F9FDABE,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,-87.696688,41.899180,-87.672178,casual,NaN,NaN,9.0,1.266479,15.466667,15.616667,37.5,0.0,55.02,0.39,False,6.0,3.0,1,1.0,2021-01-09,True
3,3,4D518C65E338D070,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,-87.696716,41.899149,-87.672177,casual,NaN,NaN,9.0,1.268119,15.466667,15.616667,37.5,0.0,55.02,0.39,False,6.0,3.0,1,1.0,2021-01-09,True
4,4,9D08A3AFF410474D,classic_bike,1/24/2021 15:56,1/24/2021 16:07,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,casual,NaN,NaN,11.0,1.266101,15.933333,16.116667,34.1,0.0,65.50,0.00,False,6.0,3.0,0,1.0,2021-01-24,True


In [207]:
def is_rnd(startid, endid):
    return startid == endid


df["is_round"] = df.apply(lambda x: is_rnd(x["start_station_id"], x["end_station_id"]), axis=1)

In [216]:
X = df[["start_hour", "mem_cas_bool", "bin_lat", "bin_lng", "ride_type_int", "weekend_or_holiday"]]

In [217]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

mms = MinMaxScaler()

X_new = mms.fit_transform(X)

lr1 = LogisticRegression(random_state=0, solver='saga', penalty="none")
cv_results1 = cross_validate(lr1, X_new, df["is_round"], scoring="roc_auc", cv=5)

cv_results1["test_score"].mean()

0.7540597460544646

In [210]:
len(df[df["is_round"] == 0]) / len(df[df["is_round"] == 1])

5.3807357401428435

In [213]:
df[["start_hour", "mem_cas_bool", "ride_type_int", "bin_lat", "bin_lng","is_round", "weekend_or_holiday"]].corr()

,start_hour,mem_cas_bool,ride_type_int,bin_lat,bin_lng,is_round,weekend_or_holiday
start_hour,1.000000,-0.061861,0.044063,0.001446,0.019602,0.023307,-0.064858
mem_cas_bool,-0.061861,1.000000,-0.197699,0.041031,-0.031302,-0.208569,-0.122527
ride_type_int,0.044063,-0.197699,1.000000,0.006794,0.034475,0.224054,0.047455
bin_lat,0.001446,0.041031,0.006794,1.000000,-0.528711,-0.053311,0.026367
bin_lng,0.019602,-0.031302,0.034475,-0.528711,1.000000,-0.024993,0.032241
is_round,0.023307,-0.208569,0.224054,-0.053311,-0.024993,1.000000,0.036018
weekend_or_holiday,-0.064858,-0.122527,0.047455,0.026367,0.032241,0.036018,1.000000
